In [16]:
import os
import CONFIG
from pprint import pprint
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


os.environ["OPENAI_API_KEY"] = CONFIG.OPENAI_API_KEY
llm_name = "gpt-3.5-turbo"
llm_name = "gpt-4-1106-preview"

In [17]:
persist_directory = "/Users/dardonacci/Documents/dardonacci/2-Code/metabrain/vector_stores/hummingbot/chroma/"

embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_scripts",
    persist_directory=persist_directory,
    embedding_function=embedding
)

# Prompts

In [18]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. You need to create the full code for the script that the user is asking for. If you think that more information is needed, ask the user for it.
{context}
Question: {question}
Helpful Answer:"""
prompt = PromptTemplate.from_template(template)

In [20]:
llm = ChatOpenAI(model_name=llm_name, temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_type="mmr", k=5, fetch_k=10),
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [21]:
query = """I would like to create a Directional Strategy using Hummingbot using the MACDBB controller. Can you the script for me and suggest values for the indicators?
"""

In [22]:
result = qa_chain({"query": query})

In [27]:
pprint(result["result"])

('Based on the context provided, it seems you want to create a directional '
 'trading strategy using the MACDBB controller in Hummingbot. Below is a '
 'script that sets up a directional strategy using the MACDBB controller. '
 'Please note that the values for the MACD and Bollinger Bands indicators are '
 'just examples, and you may need to adjust them based on your trading '
 'preferences and backtesting results.\n'
 '\n'
 '```python\n'
 'from decimal import Decimal\n'
 'from hummingbot.connector.connector_base import ConnectorBase\n'
 'from hummingbot.core.data_type.common import OrderType\n'
 'from hummingbot.smart_components.controllers.macd_bb_v1 import MACDBBV1, '
 'MACDBBV1Config\n'
 'from '
 'hummingbot.smart_components.strategy_frameworks.directional_trading.directional_trading_executor_handler '
 'import (\n'
 '    DirectionalTradingExecutorHandler,\n'
 ')\n'
 'from hummingbot.strategy.script_strategy_base import ScriptStrategyBase\n'
 'from typing import Dict\n'
 '\n'
 'cl

In [31]:
len(result["source_documents"])

4